In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torchvision.models import ResNet50_Weights
import torch.optim as optim
from torch.optim import lr_scheduler
import tqdm
import PIL.Image

#PIL.Image.MAX_IMAGE_PIXELS = 93312000000000
# tutti import + devo calcolare max image pixels

In [7]:
#############elaborazione dati
input_path = "C:\\Users\\roach\\Desktop\\archive (1)\\Hair Diseases - Final" #This is the path where the files .png are located da cambiare
BATCH_SIZE = 8
# la cartella di input va organizzata con cartella test e cartella train, ognuna deve avere tutte le classi, inclusa quelle di immagini con capelli belli senza malatie
data_transforms = transforms.Compose([
    transforms.Resize((512,512)), 
    transforms.ToTensor()
    ])#ogni immagine diventa un tensore che sarebbe un array

train_ds = datasets.ImageFolder(os.path.join(input_path ,'train'), data_transforms)
val_ds = datasets.ImageFolder(os.path.join(input_path , 'val'), data_transforms)# mario gay deve essere train e test

dataloaders = {
    'train': torch.utils.data.DataLoader(train_ds,
                                batch_size=BATCH_SIZE, shuffle = True), #si definisce la quantità del batch, lo shuffle meschia tutti i dati (capa buona e malamenta)
    'validation': torch.utils.data.DataLoader(val_ds,
                                batch_size=BATCH_SIZE, shuffle = True)
}

In [8]:
# Selecting the device on apple silicon
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

#We use the resnet50 model for this problem
model = models.resnet50(weights=ResNet50_Weights.DEFAULT).to(device) #qua usa una reNet50 già allenata, se voglio allenarla da zero devo togliere weights

# Replacing the last fully-connected layer
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 10),
               nn.Softmax(dim=1)).to(device)#fine della rete 

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

Using device: cuda


In [9]:
# Define the optimizer

optimizer = optim.Adam(model.parameters(), lr=0.02)


# Define the scheduler

# Adjust patience, factor, and other parameters based on your preference

scheduler = lr_scheduler.ReduceLROnPlateau(

    optimizer, mode="min", patience=2, factor=0.1, verbose=True
)


# Create a file to save the training and validation losses

loss_file_path = "C:\\Users\\roach\\Desktop\\resnet50cose\\losses.csv"

with open(loss_file_path, "w") as loss_file:

    loss_file.write("epoch,train_loss,train_acc,validation_loss,validation_acc\n")



def train_model_with_scheduler(

    model, criterion, optimizer, scheduler, num_epochs, loss_file_path

):

    for epoch in range(num_epochs):

        print("Epoch {}/{}".format(epoch + 1, num_epochs))
        train_loss, train_acc, val_loss, val_acc = 0, 0, 0, 0 

        for phase in ["train", "validation"]:

            if phase == "train":
                model.train()

            else:

                model.eval()


            count = 0

            running_loss = 0.0

            running_corrects = 0


            for inputs, labels in tqdm.tqdm(dataloaders[phase]):

                inputs = inputs.to(device)

                labels = labels.to(device).long()
                outputs = model(inputs)

                loss = criterion(outputs, labels)


                if phase == "train":

                    optimizer.zero_grad()

                    loss.backward()

                    optimizer.step()


                preds = torch.argmax(outputs, dim=1)


                running_loss += loss.item() * inputs.size(0)

                running_corrects += torch.sum(preds == labels.data)

                count += 1


            # Step the scheduler based on the validation loss

            if phase == "validation":

                scheduler.step(running_loss)

            torch.save(
                model.state_dict(),
                "C:\\Users\\roach\\Desktop\\resnet50cose\\" + str(epoch) + " " + phase+".pth",
            )


            epoch_loss = running_loss / (count * BATCH_SIZE)
            
            epoch_acc = running_corrects.float() / (count * BATCH_SIZE)

            if phase == "train":
                train_loss = epoch_loss
                train_acc = epoch_acc
            else:
                val_loss = epoch_loss
                val_acc = epoch_acc

            print("{} loss: {:.4f}, acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            print(

                "Current learning rate: {:.6f}".format(optimizer.param_groups[0]["lr"])
            )


            # Save losses and accuracies to the file

        with open(loss_file_path, "a") as loss_file:

            loss_file.write(

                "{},{:.6f},{:.6f},{:.6f},{:.6f} \n".format(
                    epoch,
                    train_loss,
                    train_acc,
                    val_loss,
                    val_acc
                )
            )

    return model



# Train the model with the scheduler

model_trained = train_model_with_scheduler(

    model, criterion, optimizer, scheduler, num_epochs=40, loss_file_path=loss_file_path
)
model = None

Epoch 1/40


100%|██████████| 1200/1200 [25:59<00:00,  1.30s/it]


train loss: 2.3160, acc: 0.1105
Current learning rate: 0.020000


100%|██████████| 150/150 [00:52<00:00,  2.83it/s]


validation loss: 2.2793, acc: 0.1367
Current learning rate: 0.020000
Epoch 2/40


...

train loss: 2.3612, acc: 0.1000
Current learning rate: 0.000002


100%|██████████| 150/150 [00:49<00:00,  3.05it/s]


Epoch 00018: reducing learning rate of group 0 to 2.0000e-07.
validation loss: 2.3612, acc: 0.1000
Current learning rate: 0.000000
Epoch 19/40


 28%|██▊       | 337/1200 [09:04<23:13,  1.61s/it]


KeyboardInterrupt: 